<a href="https://colab.research.google.com/github/heng112/Python_memo/blob/main/python_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ライブラリより読み込み
# from preprocess.load_data.data_loader import load_hotel_reserve
# customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()
import pandas as pd
# CSVより読み込み
reserve_tb = pd.read_csv('reserve.csv', encoding='UTF-8')

## 抽出


loc/lioc/ix　の3つの関数

.loc[行, 列名]　（列名が推奨）

.iloc[行, 列番号]　(アンチパターン　できれば避ける)

.ix[行, 列名]　又は　[行, 列番号]　（わかりづらい　非推奨）

In [ ]:
reserve_tb[['reserve_id','hotel_id','customer_id','reserve_datetime','checkin_date','checkin_time','checkout_date']]

In [ ]:
reserve_tb.loc[:, ['reserve_id','hotel_id','customer_id','reserve_datetime','checkin_date','checkin_time','checkout_date']]

### 列を削除する方法

drop関数

.drop([列or行名], axis=0か1 , inplace=FalseかTrue)

axis　→　行(0)か列(1)を指定

inplace 　→　書き換えするかどうか

In [ ]:
reserve_tb.drop(['people_num', 'total_price'], axis=1, inplace=True)

### 条件指定

qurey関数　（推奨）


DataFrameを利用した条件式　（非推奨）

In [ ]:
reserve_tb.query('"2016-10-13" <= checkout_date <= "2016-10-14"')

### サンプリング

sample関数　推奨

.sample(引数)

引数

　franc = 0.5　50%の割合

  n = 100　100件


In [ ]:
# reserve_tbから50%サンプリング
reserve_tb.sample(frac=0.5)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
2878,r2879,h_102,c_724,2018-04-11 18:32:32,2018-04-22,10:30:00,2018-04-25,1,18000
324,r325,h_253,c_76,2016-03-25 20:45:08,2016-04-15,11:30:00,2016-04-18,2,31200
3964,r3965,h_98,c_985,2017-02-09 17:13:03,2017-02-26,12:00:00,2017-03-01,3,170100
2853,r2854,h_201,c_720,2016-09-16 14:39:02,2016-09-19,10:30:00,2016-09-21,3,53400
1805,r1806,h_233,c_447,2017-10-20 14:29:31,2017-11-10,11:00:00,2017-11-13,3,80100
...,...,...,...,...,...,...,...,...,...
1627,r1628,h_252,c_399,2017-01-17 06:42:38,2017-01-23,11:00:00,2017-01-26,4,266400
2988,r2989,h_244,c_754,2016-11-14 07:32:34,2016-12-07,11:00:00,2016-12-10,3,89100
1240,r1241,h_174,c_303,2016-10-07 06:37:27,2016-10-21,10:30:00,2016-10-24,4,264000
554,r555,h_3,c_131,2016-02-10 00:25:39,2016-02-22,09:30:00,2016-02-24,4,330400


In [ ]:
# reserve_tb['customer_id'].unique()は、重複を排除したcustomer_idを返す
# sample関数を利用するためにpandas.Series(pandasのリストオブジェクト)に変換
# sample関数によって、顧客IDをサンプリング
target = pd.Series(reserve_tb['customer_id'].unique()).sample(frac=0.5)

# isin関数によって、customer_idがサンプリングした顧客IDのいずれかに一致した行を抽出
reserve_tb[reserve_tb['customer_id'].isin(target)]

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
16,r17,h_115,c_3,2016-05-10 12:20:32,2016-05-17,10:00:00,2016-05-19,2,164000
17,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400
18,r19,h_23,c_3,2017-01-11 22:54:09,2017-02-08,10:00:00,2017-02-10,3,390600
19,r20,h_292,c_3,2017-02-23 07:10:30,2017-03-03,11:00:00,2017-03-04,2,18200
20,r21,h_153,c_3,2017-04-06 18:12:10,2017-04-16,09:00:00,2017-04-19,3,126900
...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000


## 集約

主に二つ

groupByで集約関数を利用

Window関数

In [ ]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

cus_cnt_tb = \
  reserve_tb.groupby('hotel_id')['customer_id'].nunique().reset_index()

cus_cnt_tb.columns = ['hotel_id', 'cus_cnt']
pd.merge(rsv_cnt_tb, cus_cnt_tb, on='hotel_id')

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


In [ ]:
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'reserve_id': 'count', 'customer_id': 'nunique'})

# reset_index関数によって、列番号を振り直す（inplace=Trueなので、直接resultを更新）
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']

### 合計値の算出

sum関数


In [ ]:
result = reserve_tb \
    .groupby(['hotel_id', 'people_num']['total_price'])\
    .sum().reset_index()

TypeError: ignored

In [ ]:
result = reserve_tb \
  .groupby(['hotel_id', 'people_num'])['total_price'] \
  .sum().reset_index()
result

,hotel_id,people_num,total_price
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200
...,...,...,...
1154,h_98,3,793800
1155,h_98,4,453600
1156,h_99,1,179200
1157,h_99,2,448000
